# Imports

In [ ]:
import tensorflow as tf
from tensorflow.keras.applications.vgg19 import VGG19, preprocess_input
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Flatten, Dense, Dropout
from tensorflow.keras.models import load_model
import numpy as np
import o
import warnings
warnings.filterwarnings("ignore")



# Building Pretrained model

In [3]:
def build_vgg19():
    base = VGG19(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
    for layer in base.layers:
        layer.trainable = False
    x = Flatten()(base.output)
    x = Dense(256, activation='relu')(x)
    x = Dropout(0.5)(x)
    out = Dense(1, activation='sigmoid')(x)
    return Model(base.input, out)

# Data Generators

In [4]:
import shutil
from sklearn.model_selection import train_test_split

base_dir = "/kaggle/input/cifake-real-and-ai-generated-synthetic-images/train"
val_dir = "/kaggle/working/validation"

for label in ['REAL', 'FAKE']:
    os.makedirs(os.path.join(val_dir, label), exist_ok=True)
    files = os.listdir(os.path.join(base_dir, label))
    train_files, val_files = train_test_split(files, test_size=0.2, random_state=42)

    for fname in val_files:
        src = os.path.join(base_dir, label, fname)
        dst = os.path.join(val_dir, label, fname)
        shutil.copyfile(src, dst)


In [5]:
train_gen = ImageDataGenerator(preprocessing_function=preprocess_input).flow_from_directory(
    '/kaggle/input/cifake-real-and-ai-generated-synthetic-images/train',
    target_size=(224, 224), batch_size=32, class_mode='binary', shuffle=True)

val_gen = ImageDataGenerator(preprocessing_function=preprocess_input).flow_from_directory(
    '/kaggle/working/validation',
    target_size=(224, 224), batch_size=32, class_mode='binary', shuffle=False)

test_gen = ImageDataGenerator(preprocessing_function=preprocess_input).flow_from_directory(
    '/kaggle/input/cifake-real-and-ai-generated-synthetic-images/test',
    target_size=(224, 224), batch_size=32, class_mode='binary', shuffle=False)

Found 100000 images belonging to 2 classes.
Found 20000 images belonging to 2 classes.
Found 20000 images belonging to 2 classes.


# FGSM Attack Function and PGD Attack Function

In [6]:
@tf.function
def fgsm_attack(model, images, labels, epsilon=0.05):
    with tf.GradientTape() as tape:
        tape.watch(images)
        predictions = model(images)
        loss = tf.keras.losses.BinaryCrossentropy()(labels, predictions)
    gradient = tape.gradient(loss, images)
    signed_grad = tf.sign(gradient)
    adv_images = images + epsilon * signed_grad
    return tf.clip_by_value(adv_images, 0, 1)

# ✅ PGD Attack Function
@tf.function
def pgd_attack(x, y, model, loss_fn, epsilon=0.03, alpha=0.007, iters=10):
    x_adv = tf.identity(x)

    for i in range(iters):
        with tf.GradientTape() as tape:
            tape.watch(x_adv)
            prediction = model(x_adv, training=False)
            loss = loss_fn(y, prediction)

        grad = tape.gradient(loss, x_adv)
        signed_grad = tf.sign(grad)
        x_adv = x_adv + alpha * signed_grad
        x_adv = tf.clip_by_value(x_adv, x - epsilon, x + epsilon)
        x_adv = tf.clip_by_value(x_adv, 0.0, 1.0)  # ensure valid pixel range

    return x_adv


#  Train and Save Baseline Model

In [6]:
# Paths to save model and weights
weights_path_baseline = '/kaggle/working/vgg19_baseline_cifake.weights.h5'
model_path_baseline   = '/kaggle/working/vgg19_baseline_cifake_model.h5'
#Kaggle
if os.path.exists('/kaggle/input/vgg19_baseline_cifake/tensorflow2/default/1/vgg19_baseline_cifake_model.h5'):
    print("Loading saved model...")
    baseline_model = load_model('/kaggle/input/vgg19_baseline_cifake/tensorflow2/default/1/vgg19_baseline_cifake_model.h5')
#Colab
# if os.path.exists(model_path_baseline):
#     print("Loading saved model...")
#     baseline_model = load_model(model_path_baseline)
else:
    print("Training baseline model...")
    baseline_model = build_vgg19()
    baseline_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    baseline_model.fit(train_gen, epochs=3, validation_data=val_gen)
    baseline_model.save_weights(weights_path_baseline)
    baseline_model.save(model_path_baseline)
    print("Baseline model trained and saved.")

print("\nEvaluating Baseline Model on Clean Test Set:")
baseline_clean_acc = baseline_model.evaluate(test_gen)
print("Baseline Clean Accuracy:", baseline_clean_acc)


Loading saved model...


I0000 00:00:1744727042.531207      31 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 13942 MB memory:  -> device: 0, name: Tesla T4, pci bus id: 0000:00:04.0, compute capability: 7.5
I0000 00:00:1744727042.531875      31 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 13942 MB memory:  -> device: 1, name: Tesla T4, pci bus id: 0000:00:05.0, compute capability: 7.5



Evaluating Baseline Model on Clean Test Set:


/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()
I0000 00:00:1744727047.967304     113 service.cc:148] XLA service 0x79de8c00c100 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1744727047.969142     113 service.cc:156]   StreamExecutor device (0): Tesla T4, Compute Capability 7.5
I0000 00:00:1744727047.969163     113 service.cc:156]   StreamExecutor device (1): Tesla T4, Compute Capability 7.5
I0000 00:00:1744727048.219918     113 cuda_dnn.cc:529] Loaded cuDNN version 90300


  1/625 ━━━━━━━━━━━━━━━━━━━━ 2:11:37 13s/step - accuracy: 0.9375 - loss: 0.3287

I0000 00:00:1744727059.586375     113 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


625/625 ━━━━━━━━━━━━━━━━━━━━ 168s 249ms/step - accuracy: 0.8809 - loss: 0.2490
Baseline Clean Accuracy: [0.20891635119915009, 0.913349986076355]


# FGSM Vulnerability Evaluation

In [7]:
x_test_sample, y_test_sample = next(test_gen)
x_test_tensor = tf.convert_to_tensor(x_test_sample, dtype=tf.float32)
y_test_tensor = tf.convert_to_tensor(y_test_sample, dtype=tf.float32)

x_fgsm_test = fgsm_attack(baseline_model, x_test_tensor, y_test_tensor)
print("\nEvaluating Baseline Model on FGSM-Adversarial Examples:")
baseline_fgsm_acc = baseline_model.evaluate(x_fgsm_test, y_test_sample)
print("Baseline Accuracy on FGSM-Adversarial:", baseline_fgsm_acc)


Evaluating Baseline Model on FGSM-Adversarial Examples:
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 288ms/step - accuracy: 0.1250 - loss: 1.1523
Baseline Accuracy on FGSM-Adversarial: [1.152273416519165, 0.125]


# Generate and Save Perturbed Data (FGSM + PGD)

In [9]:
import tensorflow as tf
import numpy as np
import os
from tqdm import tqdm

# Directories
base_train_dir = "/kaggle/input/cifake-real-and-ai-generated-synthetic-images/train"
perturbed_dir = "/kaggle/working/train_perturbed_only"  # All perturbed, no clean data
weights_path_baseline = "/kaggle/input/vgg19_baseline_cifake/tensorflow2/default/1/vgg19_baseline_cifake.weights.h5"

# Create directories
os.makedirs(os.path.join(perturbed_dir, 'REAL'), exist_ok=True)
os.makedirs(os.path.join(perturbed_dir, 'FAKE'), exist_ok=True)

# Load baseline model (for generating perturbations)
baseline_model = build_vgg19()
baseline_model.load_weights(weights_path_baseline)  # Pretrained weights

# Data generator (no shuffling for consistent file mapping)
train_gen = ImageDataGenerator(preprocessing_function=preprocess_input).flow_from_directory(
    base_train_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary',
    shuffle=False  # Critical for matching files to paths
)

# Attack parameters
epsilon_fgsm = 0.05
epsilon_pgd = 0.03
alpha_pgd = 0.007
iters_pgd = 10
loss_fn = tf.keras.losses.BinaryCrossentropy()

# Generate and save perturbed data
num_batches = len(train_gen)

for i in tqdm(range(num_batches)):
    x_batch, y_batch = next(train_gen)
    
    # Get indices and paths for this batch
    batch_indices = train_gen.index_array[i*32 : (i+1)*32]
    paths = [train_gen.filepaths[idx] for idx in batch_indices]

    # Split batch: 50% FGSM, 50% PGD
    split = len(x_batch) // 2

    # Generate FGSM samples
    x_fgsm = fgsm_attack(
        baseline_model, 
        tf.convert_to_tensor(x_batch[:split]), 
        tf.convert_to_tensor(y_batch[:split]), 
        epsilon_fgsm
    ).numpy()

    # Generate PGD samples
    x_pgd = pgd_attack(
        tf.convert_to_tensor(x_batch[split:]), 
        y_batch[split:], 
        baseline_model, 
        loss_fn,
        epsilon=epsilon_pgd, 
        alpha=alpha_pgd, 
        iters=iters_pgd
    ).numpy()

    # Combine and save
    x_perturbed = np.concatenate([x_fgsm, x_pgd], axis=0)

    # Save images
    for j, path in enumerate(paths):
        class_name = 'REAL' if 'REAL' in path else 'FAKE'
        filename = os.path.basename(path).replace('.jpg', f'_perturbed_{i}_{j}.jpg')
        save_path = os.path.join(perturbed_dir, class_name, filename)
        tf.keras.preprocessing.image.save_img(save_path, x_perturbed[j])

Found 100000 images belonging to 2 classes.


100%|██████████| 3125/3125 [2:14:12<00:00,  2.58s/it]  


# Train on Perturbed Data

In [8]:
perturbed_gen = ImageDataGenerator(preprocessing_function=preprocess_input).flow_from_directory(
    "/kaggle/working/train_perturbed_only",  # Directory with 100% perturbed data
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary',
    shuffle=True
)

# Initialize model (same architecture as baseline)
adv_model = build_vgg19()
adv_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train exclusively on adversarial data
adv_model.fit(perturbed_gen, epochs=3, validation_data=val_gen)

Found 100000 images belonging to 2 classes.


I0000 00:00:1744742373.180338     339 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 13942 MB memory:  -> device: 0, name: Tesla T4, pci bus id: 0000:00:04.0, compute capability: 7.5
I0000 00:00:1744742373.181059     339 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 13942 MB memory:  -> device: 1, name: Tesla T4, pci bus id: 0000:00:05.0, compute capability: 7.5


Epoch 1/3


/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()
I0000 00:00:1744742379.193533     401 service.cc:148] XLA service 0x7f47b400e220 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1744742379.199739     401 service.cc:156]   StreamExecutor device (0): Tesla T4, Compute Capability 7.5
I0000 00:00:1744742379.199771     401 service.cc:156]   StreamExecutor device (1): Tesla T4, Compute Capability 7.5
I0000 00:00:1744742379.749710     401 cuda_dnn.cc:529] Loaded cuDNN version 90300


   1/3125 ━━━━━━━━━━━━━━━━━━━━ 13:44:42 16s/step - accuracy: 0.5625 - loss: 5.8884

I0000 00:00:1744742391.808150     401 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


3125/3125 ━━━━━━━━━━━━━━━━━━━━ 910s 286ms/step - accuracy: 0.7026 - loss: 1.5068 - val_accuracy: 0.6962 - val_loss: 0.7808
Epoch 2/3
3125/3125 ━━━━━━━━━━━━━━━━━━━━ 915s 293ms/step - accuracy: 0.7626 - loss: 0.4895 - val_accuracy: 0.7009 - val_loss: 0.7069
Epoch 3/3
3125/3125 ━━━━━━━━━━━━━━━━━━━━ 918s 294ms/step - accuracy: 0.7726 - loss: 0.4654 - val_accuracy: 0.6479 - val_loss: 1.4056


# Saving the Model

In [11]:
    weights_path_adv = '/kaggle/working/vgg19_adv_cifake.weights.h5'
    model_path_adv   = '/kaggle/working/vgg19_adv_cifake_model.h5'
    
    adv_model.save_weights(weights_path_adv)
    adv_model.save(model_path_adv)

# Evaluate Robustness

In [10]:
# Evaluate on clean test data (optional)
clean_loss, clean_acc = adv_model.evaluate(test_gen, verbose=0)
print(f"Clean Test Accuracy: {clean_acc:.4f}")

# Evaluate on FGSM-attacked test data
x_test, y_test = next(test_gen)
x_fgsm_test = fgsm_attack(adv_model, tf.convert_to_tensor(x_test), tf.convert_to_tensor(y_test), 0.05)
fgsm_loss, fgsm_acc = adv_model.evaluate(x_fgsm_test, y_test, verbose=0)
print(f"FGSM Test Accuracy: {fgsm_acc:.4f} (Baseline was ~12.5%)")

Clean Test Accuracy: 0.6515
FGSM Test Accuracy: 0.7500 (Baseline was ~12.5%)
